## Use Inbuilt tools_condition for Conditional Routing

In [2]:
#Important Libraries
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode, tools_condition


In [3]:
#Initialize the LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

#Define a multiplication tool
def multiply(a: int, b: int) -> int:
    """ Multiplies two numbers."""
    
    return a * b


#Bind the LLM with tool
llm_with_tools = llm.bind_tools([multiply])

#Node that call the LLM with tools bound
def tool_calling_llm(state: MessagesState):
    """Node that calls the LLM with tools bound."""

    return{"messages": [llm_with_tools.invoke(state["messages"])]}


#Build the workflow graph
workflow = StateGraph(MessagesState)
workflow.add_node("tool_calling_llm", tool_calling_llm)
#Tool node for handling tool invokations
workflow.add_node("tools", ToolNode([multiply]))

#Define edges with conditional routing
workflow.add_edge(START, "tool_calling_llm")
#Add the conditional edge based on tool usage
workflow.add_conditional_edges("tool_calling_llm", tools_condition)
workflow.add_edge("tools", END)

#Compile the graph
graph = workflow.compile()


In [4]:
#Simulate invoking the graph
def simulate():

    user_input = {"messages": [("human", "Can you multiply 3 by 5?")]}
    result = graph.invoke(user_input)
    return result["messages"][-1].pretty_print()

print(simulate())

================================= Tool Message =================================
Name: multiply

15
None


## Custom conditional routing function example: Weather and Calculator nodes

In [7]:
from langgraph.graph import StateGraph, MessagesState, START, END

#Define a node to simulate a weather response
def weather_node(state: MessagesState):
    """Node that simulates a weather response."""
    return {"messages": ["Its sunny today!"]}

#Define a node to handle basic arithmetic calculations
def calculator_node(state: MessagesState):
    """Node that handles basic arithmetic calculations."""
    user_query = state["messages"][-1].content.lower()
    if "add" in user_query:
        numbers = [int(s) for s in user_query.split() if s.isdigit()]
        result = sum(numbers)
        return {"messages": [f"The result is {result}."]}
    return {"messages": ["I can only add numbers."]}

#Define a default node to handle unrecognized queries
def default_node(state: MessagesState):
    """Default node to handle unrecognized queries."""
    return {"messages": ["I'm sorry, I don't understand your request."]}


#Custome routing function to decide which node to route to
def routing_function(state: MessagesState):
    """Custom routing function to decide which node to route to."""
    last_message = state["messages"][-1].content.lower()
    if "weather" in last_message:
        return "weather_node"
    elif "add" in last_message or "calculate" in last_message:
        return "calculator_node"
    return "default_node"


#Build the workflow graph
workflow = StateGraph(MessagesState)
workflow.add_node("weather_node", weather_node)
workflow.add_node("calculator_node", calculator_node)
workflow.add_node("default_node", default_node)
workflow.add_node("routing_function", routing_function)

#Set up the edges for routing
workflow.add_conditional_edges(START, routing_function)
workflow.add_edge("weather_node", END)
workflow.add_edge("calculator_node", END)
workflow.add_edge("default_node", END)


#Compile the graph
app = workflow.compile()


In [8]:
#Simulate interactions with the agent
def simulate_interaction():
    while True:
        user_input = input("User: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Exiting interaction.")
            break
        input_message = {"messages": [("human", user_input)]}
        for result in app.stream(input_message, stream_mode="values"):
            result["messages"][-1].pretty_print()




#Start the simulation
simulate_interaction()            

================================ Human Message =================================

what is the weather today?
================================ Human Message =================================

Its sunny today!
================================ Human Message =================================

can you add 10 and 20?
================================ Human Message =================================

The result is 10.
================================ Human Message =================================

Can you add 10 and 20?
================================ Human Message =================================

The result is 10.
================================ Human Message =================================

Hello how are you?
================================ Human Message =================================

I'm sorry, I don't understand your request.
Exiting interaction.
